# Анализ базы данных книжного сервиса

**Цель проекта**:  анализ информации, содержащейся в базе данных сервиса для чтения книг для последующего формулирования ценностного предложения для нового продукта.

**Этапы реализации проекта:** 

[1. Загрузка библиотек, выгрузка примеров данных, изучение общей информации о таблице](#general)

[2. Подсчет книг, вышедших после 1 января 2000 года](#book)

[3. Подсчет количества обзоров и средней оценки для книг](#reviews_scores)

[4. Определение издательства с наибольшим количеством книг свыше 50 страниц](#publisher)

[5. Определение автора с самой высокой средней оценкой книг (свыше 50 оценок)](#author)

[6. Подсчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок](#users)

[7. Выводы](#conclusion)

<a id="general"> </a>
## Загрузка библиотек, выгрузка примеров данных, изучение общей информации о таблице

In [17]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [18]:
#создадим функцию, которая упростит выдачу результата запросов

def sql_query(this_query):
    return pd.io.sql.read_sql(this_query, con = engine) 

В распоряжении компании имеются **следующие таблицы**:
- `books` (информация о книгах)
- `authors` (информация об авторах)
- `publishers` (информация об издателях)
- `ratings` (информация о рейтинге)
- `reviews` (информация об обзорах)

In [19]:
#выведем первые 5 строк таблицы books

query = '''
    SELECT*
    FROM books
    LIMIT 5
    '''
sql_query(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [20]:
#выведем первые 5 строк таблицы authors

query = '''
    SELECT*
    FROM authors
    LIMIT 5
    '''
sql_query(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [21]:
#выведем первые 5 строк таблицы publishers

query = '''
    SELECT*
    FROM publishers
    LIMIT 5
    '''
sql_query(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [22]:
#выведем первые 5 строк таблицы ratings

query = '''
    SELECT*
    FROM ratings
    LIMIT 5
    '''
sql_query(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [23]:
#выведем первые 5 строк таблицы reviews

query = '''
    SELECT*
    FROM reviews
    LIMIT 5
    '''
sql_query(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

<a id="book"> </a>
## Подсчет книг, вышедших после 1 января 2000 года

Определим, сколько книг вышло после 1 января 2000 года включительно

In [24]:
#дополнительно уточним, за какой период у нас имеется информация о данных

query = '''
    SELECT MIN(publication_date), MAX(publication_date)
    FROM books
    '''
sql_query(query)

,min,max
0,1952-12-01,2020-03-31


In [25]:
#выполним запрос 

query = '''
    SELECT COUNT(book_id)
    FROM books
    WHERE publication_date >= '2000-01-01'
    '''
sql_query(query)

,count
0,821


В период с 1 января 2000 года по 31 марта 2020 года вышла 821 книга, включенная в базу данных.

<a id="reviews_scores"> </a>
## Подсчет количества обзоров и средней оценки для книг

In [26]:
#дополнительно уточним, какой у нас имеется самый высокий и самый низкий рейтинг

query = '''
    SELECT MIN(rating), MAX(rating)
    FROM ratings
    '''
sql_query(query)

,min,max
0,1,5


In [27]:
#проверим количество обзоров

query = '''
        SELECT COUNT(review_id) 
        FROM reviews
        '''

sql_query(query)

,count
0,2793


In [28]:
#выполним запрос и сделаем сортировку по рейтингу

query = '''
    SELECT b.book_id, 
        b.title,
        COUNT(DISTINCT r.review_id) as review_count,
        ROUND(AVG(rt.rating), 2) as avg_rating
    FROM books as b
    LEFT JOIN reviews as r ON b.book_id = r.book_id
    LEFT JOIN ratings as rt ON b.book_id = rt.book_id
    GROUP BY b.book_id
    ORDER BY avg_rating DESC;
    '''
sql_query(query)

,book_id,title,review_count,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


In [29]:
#выполним запрос и сделаем сортировку по количеству ревью 

query = '''
    SELECT b.book_id, 
        b.title,
        COUNT(DISTINCT r.review_id) as review_count,
        ROUND(AVG(rt.rating), 2) as avg_rating
    FROM books as b
    LEFT JOIN reviews as r ON b.book_id = r.book_id
    LEFT JOIN ratings as rt ON b.book_id = rt.book_id
    GROUP BY b.book_id
    ORDER BY review_count DESC, avg_rating DESC;
    '''
sql_query(query)

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Для каждой книги получено количество ревью и средний рейтинг. В базе данных наилучшие книги получили средний рейтинг равный пяти, наихудшая - 1.5. Если сделать сортировку по количеству ревью, то максимальные отзывы получила такая книга, как "Сумерки" ('Twilight', 7 отзывов при среднем рейтинге 3.66).

<a id="publisher"> </a>
## Определение издательства с наибольшим количеством книг свыше 50 страниц

In [30]:
#выполним запрос 

query = '''
    WITH b AS
        (SELECT book_id, publisher_id
        FROM books
        WHERE num_pages > 50)
    
    
    SELECT p.publisher_id, 
           p.publisher, 
           COUNT(b.book_id) AS books_number
    FROM publishers as p
    LEFT JOIN b ON p.publisher_id = b.publisher_id
    GROUP BY p.publisher_id
    ORDER BY books_number DESC
    LIMIT 1;
    '''
sql_query(query)

,publisher_id,publisher,books_number
0,212,Penguin Books,42


Большое всего книг толщиной от 50 страниц, включенных в базу данных, выпустило британское издательство Penguin Books, а именно - 42 книги. 

<a id="author"> </a>
## Определение автора с самой высокой средней оценкой книг (свыше 50 оценок)

In [31]:
#выполним запрос 

query = '''
    WITH rt AS 
        (SELECT b.book_id, COUNT(r.rating_id) as cnt
        FROM books as b
        RIGHT JOIN ratings as r ON b.book_id = r.book_id
        GROUP BY b.book_id
        HAVING COUNT(r.rating_id) >= 50),
        
    br AS
        (SELECT 
            bk.book_id, 
            bk.author_id,
            r.rating
        FROM books as bk
        RIGHT JOIN ratings as r ON bk.book_id = r.book_id)
    
    SELECT a.author_id,
           a.author, 
           AVG(br.rating) AS avg_rating
    FROM authors as a
    LEFT JOIN br ON a.author_id = br.author_id
    RIGHT JOIN rt ON br.book_id = rt.book_id
    GROUP BY a.author_id
    ORDER BY avg_rating DESC
    LIMIT 1;
    '''
sql_query(query)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


Наилучший средний рейтинг - 4.287 - получили работы писательницы Джоан Роулинг и иллюстратора Мэри Грандпре: именно книги о волшебном мире Гарри Поттера способствовали столь высокой оценке их творчества. 

<a id="users"> </a>
## Подсчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок

In [32]:
#выполним запрос 

query = '''
    SELECT ROUND(AVG(rv_cnt)) as avg_count
    FROM 
        (SELECT username, COUNT(review_id) as rv_cnt
            FROM reviews
            GROUP BY username
            HAVING username IN 
                (SELECT username
                FROM ratings
                GROUP BY username
                HAVING COUNT(rating_id) >= 50)) as av;
    '''

sql_query(query)

,avg_count
0,24.0


На каждого наиболее активного пользователя с точки зрения оценок книг (от 50 и выше) приходится в среднем 24 обзора.

<a id="conclusion"> </a>
## Выводы

По итогам проведенного исследования получены следующие **результаты**.

   1) Книжный сервис имеет в своей базе 821 книгу, вышедшую в XXI веке.
   
   2) Книжный сервис позволяет оценивать книги по шкале от 0 до 5. Наилучшие книги получили средний рейтинг 5.0, наихудшая - 1.5. Максимальное число отзывов (7) получила нашумевшая юношеская сага "Сумерки". В числе лидеров по числу обзоров также романы о Гарри Поттере.
   
   3) Наибольшее число книг в сервисе объемом от 50 страниц принадлежат издательству Penguin Books - 42 книги;
    
   4) Наивысший средний рейтинг получил авторский дуэт, работавший над вселенной Гарри Поттерав в лице писательницы Джоан Роулинг и иллюстратора Мэри Грандпре: они получили наивысший средний рейтинг, равный 4.287 баллов, среди авторов, работы которых были оценены не менее 50 пользователями.
   
   5) Самые активные пользователи, которые прочитали и оценили не менее 50 книг, написали в среднем по 24 обзора. 